# Loading the training data set

Note: Download the dataset from kaggle for the below code to work and place it in the same directory.

In [22]:
import cv2
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from skimage import exposure
from sklearn.preprocessing import StandardScaler, MinMaxScaler

train_images_files     = 'train_ims'
train_label_file = 'train.csv'
# test_images_files      = 'test_ims'
# test_label_file      = 'test.csv'

def scale(xs):
    stdsc = StandardScaler()
    return stdsc.fit_transform(xs)

def flatten_images(images):
    """
    Every pixel has 3 rgb values.
    Treat every rgb value as a single attribute. Therefore, for every image, there will be 32 x 32 x 3 attributes
    """
    flattened_images = images.reshape(len(images), -1)
    return flattened_images

def normalize(xs):
    
    mms = MinMaxScaler()
    xs_norm    = mms.fit_transform(xs)
    
    return xs_norm

def load_data(dataset_path, csv_path):
    '''
    Load the image data and labels.
    '''
    
    df = pd.read_csv(csv_path)
    images = []
    labels = []
    
    for index, row in df.iterrows():
        image_name = row['im_name']
        label      = row['label']
        
        image_path  = os.path.join(dataset_path, image_name)
        image       = cv2.imread(image_path)
        images.append(image)
        labels.append(label)
        
    return np.array(images), np.array(labels)



xs, ys  = load_data(train_images_files, train_label_file)
xs      = flatten_images(xs)
xs      = normalize(xs)
x_train, x_test, y_train, y_test = train_test_split(
    xs, 
    ys, 
    test_size=0.2, 
    random_state=42,
    stratify=ys 
)

In [23]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(40000, 3072) (40000,) (10000, 3072) (10000,)


In [12]:
from skimage import exposure, feature

def compute_hog_features(images):
    '''
    Compute HOG features for a list of images.
    '''
    hog_features = []
    
    for image in images:
        # Ensure the image is in uint8 format
        if image.dtype != np.uint8:
            image = (image * 255).astype(np.uint8)  # Scale back to 0-255 if it's float

        # Convert to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Compute HOG features
        hog_feature = feature.hog(gray_image, 
                                  orientations=9, 
                                  pixels_per_cell=(8, 8),
                                  cells_per_block=(2, 2), 
                                  block_norm='L2-Hys', 
                                  visualize=False)
        
        hog_features.append(hog_feature)
    
    return np.array(hog_features)

# Compute HOG features for training and testing sets
hog_train_features = compute_hog_features(x_train)
hog_test_features = compute_hog_features(x_test)

print("HOG features shape for training set:", hog_train_features.shape)
print("HOG features shape for testing set:", hog_test_features.shape)

error: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::(anonymous namespace)::CvtHelper<cv::impl::(anonymous namespace)::Set<3, 4, -1>, cv::impl::(anonymous namespace)::Set<1, -1, -1>, cv::impl::(anonymous namespace)::Set<0, 2, 5>, cv::impl::(anonymous namespace)::NONE>::CvtHelper(cv::InputArray, cv::OutputArray, int) [VScn = cv::impl::(anonymous namespace)::Set<3, 4, -1>, VDcn = cv::impl::(anonymous namespace)::Set<1, -1, -1>, VDepth = cv::impl::(anonymous namespace)::Set<0, 2, 5>, sizePolicy = cv::impl::(anonymous namespace)::NONE]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [19]:
# SVM
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

svm_model = SVC(kernel='rbf', C=5, random_state=1)
svm_model.fit(x_train, y_train)

y_pred = svm_model.predict(x_test)

# Step 6: Evaluate the Model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

ValueError: y should be a 1d array, got an array of shape (100, 3072) instead.